# 7. Generate Search Result

In this notebook, we are going to generate the search results using the 3 ranking method we have developed.  
The results are output as tsv files for our raters to review and give scores.

In [1]:
import os
import faiss
import openai
import pandas as pd
import numpy as np
import altair as alt
import configparser
import session_info

import warnings
warnings.filterwarnings('ignore')

### Prepare required resources

In [2]:
# This dataframe is created with the amazon_reviews_us_Apparel_v1_00.tsv.gz dataset with following filtering criteria:
# 1. Year 2015
# 2. Products with 10 - 14 reviews
# 3. review_lenght > 10

df_apparel = pd.read_csv('../resources/data/apparel_10to14.tsv.gz', sep='\t', compression='gzip')
cols = ['product_id', 'review_id', 'star_rating', 'product_title', 'review_body']
df_apparel = df_apparel[cols]
df_apparel.head(3)

,product_id,review_id,star_rating,product_title,review_body
0,B00001QHXX,R2HBUQ97RV5JVR,3.0,Richard Nixon Mask,I got this mask for a company party and it fre...
1,B00001QHXX,RHCH92YNAS282,4.0,Richard Nixon Mask,"Nice mask, will have to do some fitting work. ..."
2,B00001QHXX,R1OHYB07D0WE35,5.0,Richard Nixon Mask,"Even though its a bit large, I can't help but ..."


In [3]:
# Load the pre-created FAISS index
faiss_index = faiss.read_index('../resources/binary/apparel_10to14_review_cosine.faissindex')

In [4]:
# Load your API key from an environment variable or secret management service
config = configparser.ConfigParser()
config.read('nes.ini')
openai.api_key = config['OpenAI']['api_key']

### Define helper functions

In [5]:
# Helper function to create the query embedding. Make sure to use the same model as what we used to created the product embedding
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   
   return np.array(openai.Embedding.create(input = [text], model=model)['data'][0]['embedding'], dtype='float32').reshape(1, -1)

In [6]:
def search_with_original_query(df, faiss_index, query_embedding, num_of_records=100):
    # we need to normalize the question embedding in order to use cosine similarity to search 
    faiss.normalize_L2(query_embedding)

    # distance is the correspnding distance
    # result_idx is the index of the input array, hence the index of the dataframe (if the dataframe index is reset which starts with 0)
    distance, result_idx = faiss_index.search(query_embedding, k=num_of_records)

    # use the return index to create the result dataframe
    df_result = df.iloc[result_idx.squeeze().tolist()]
    # add Distance to the result dataframe
    df_result['distance'] = distance.T

    df_result = df_result.sort_values(by='distance', ascending=True)
    
    return df_result, result_idx

In [7]:
def generate_opposite_query(orignal_query=''):

    prompt="""
        You are an English teacher. You need to find every single ADJECTIVE from the sentences delimited by triple backquotes below.
        Then, you transform every adjective into its antonym.
        Finally, give the dictionary meaning for each antonym.
        Below are two examples. You need to comlete the third one. 
        

        Text 1: Kids flip flops for girl, cute, good fit, comfortable and durable, low price
        Output 1: Artless means without guile or deception. Unsuited means not proper or fitting for something. Uncomfortable means causing discomfort.  Fragile means easily broken. Costly means expensive.
        ## 
        Text 2: Long sleeve shirts for men. Wrinkle-free, thick but breathable and slim fit
        Output 2: Short means having little length. Crinkle means to form many short bends or ripples. Thin means measuring little in cross section or diameter. Airtight means impermeable to air or nearly so. Wide means having a greater than usual measure across
        ##
        Text 3:  ```{}```
        Output 3:
    """

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt.format(orignal_query),
        temperature=0,
        max_tokens=1000,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )

    return response['choices'][0]['text']

In [8]:
def search_with_opposite_query(df, faiss_index, opposite_query_embedding, original_query_result_index, num_of_records=100):

    faiss.normalize_L2(opposite_query_embedding)

    # we want to make sure the opposite query only compare against the texts found by the original query 
    id_selector = faiss.IDSelectorArray(original_query_result_index.shape[1], faiss.swig_ptr(original_query_result_index))
    filtered_distances, filtered_indices = faiss_index.search(opposite_query_embedding, k=num_of_records, params=faiss.SearchParametersIVF(sel=id_selector))

    df_opposite_result = df.iloc[filtered_indices.squeeze().tolist()]
    df_opposite_result['distance'] = filtered_distances.T

    df_opposite_result = df_opposite_result.sort_values(by='distance', ascending=False)

    return df_opposite_result

In [9]:
def get_reconcile_result(df_result_original, df_result_opposite):

    df_reconcile_result = df_result_original.merge(df_result_opposite[['review_id', 'distance']], 
                            left_on='review_id', right_on='review_id', how='left', suffixes=('_original', '_opposite'))
    
    # Using Dot Product FAISS Index with L2 normaliztion, the returning result is Cosine Similiarty, rather than Distance.
    # There will turn the Cosine Similarity to Distance 
    df_reconcile_result['distance_original'] = 1 - df_reconcile_result['distance_original']
    df_reconcile_result['distance_opposite'] = 1 - df_reconcile_result['distance_opposite']

    df_reconcile_result = df_reconcile_result.sort_values(by='distance_original', ascending=True).reset_index(drop=True)

    return df_reconcile_result

In [10]:
# Helper function to clip the distance_opposite

def clip_distance_opposite(df, clipping=0.5):
    df = df.sort_values(by='distance_opposite', ascending=False).reset_index(drop=True)
    # Flatten the first n% with distance_opposite sorted in descending order  
    quantile_value = df['distance_opposite'].quantile(q=(1-clipping))
    df.loc[0:(clipping * len(df)), ['distance_opposite']] = quantile_value

    return df

In [11]:
# Helper function to calculate adjsuted distance using the distance_oppsite as a penalty term

def cal_adjusted_distance(df, k=0.5):
    df['distance_adjusted'] = df.apply(lambda row: row['distance_original'] + (k * 1/row['distance_opposite']), axis='columns')
    
    return df 


In [12]:
# Helper function to calculate review level similarity scores

def cal_review_similarity_score(df):
    # find max of distance_adjusted  
    max_distance_adjusted  = df['distance_adjusted'].max()
    # normalized adjusted distance then subtract from 1 to calculate the similarity score 
    df['similarity_score'] = df['distance_adjusted'].apply(lambda x: 1 - x / max_distance_adjusted)

    return df 

In [13]:
# Helper function to calculate the product level similarity scores 

def cal_product_similarity_score(df, method='discount_reward'):

    if method == 'average':
        df_temp = df.groupby('product_id')['similarity_score'].mean()
        df_temp = df_temp.to_frame(name='product_similarity_score').reset_index()
        df = pd.merge(df, df_temp, left_on='product_id', right_on='product_id')
    else:
        df_grouped = df.groupby(by='product_id')

        for name, data in df_grouped:
            data = data.sort_values('similarity_score', ascending=False)
            scores = []
            for cnt, (index, row) in enumerate(data.iterrows()):
                discounted_score = row['similarity_score'] / pow(2, cnt)
                scores.append(discounted_score)
            df.loc[data.index, 'product_similarity_score'] = sum(scores)
        
    df = df.sort_values(by=['product_similarity_score', 'similarity_score'], ascending=[False, False])

    return df

In [14]:
def get_final_search_result(df_reconcile_result, clipping=0, weight=0, method='discount_reward'):
    """
    Method 1:   Calculate product level similarity score by Average
                clipping=0, weight=0, method='average'

    Method 2:   Calculate product level similarity score by Discount Reward
                clipping=0, weight=0, method='discount_reward' 

    Method 3:   Calcuate product level similarity score by Discount Reward with Adjustment by Opposite Query 
                clipping=0.1, weight=0.5, method='discount_reward
    """

    df_copy = df_reconcile_result.copy()

    df_copy = clip_distance_opposite(df_copy, clipping=clipping)
    df_copy = cal_adjusted_distance(df_copy, k=weight)
    df_copy = cal_review_similarity_score(df_copy)
    df_copy = cal_product_similarity_score(df_copy, method=method)

    # re-arrange columns for output
    cols = ['product_id', 'review_id', 'star_rating', 'distance_original', 'distance_opposite',
            'distance_adjusted', 'similarity_score', 'product_similarity_score', 'product_title', 'review_body']

    df_copy = df_copy[cols]

    return df_copy

### Geneate results

We have created 9 testing queries for generate the search results

In [15]:
queries = {
            "Q1" : "Long thin cotton socks for men, need to be breathable, even feeling cool for summer time.",
            "Q2" : "Wrinkle free chiffon blouse, sleek style, long sleeve, slim fit, with comfortable inside layer",
            "Q3" : "DC or Marvel superhero costumes for adults. Comes with a mask.  Deluxe and high quality. Comfortable to wear. Washable and durable.",
            "Q4" : "Baby footed bodysuit, made with soft and comfortable material, keeps the baby snug and warm, washed and dried well without shrinking", 
            "Q5" : "Women's knee-length A-line dress, looks cutie, made with high quality and relatively thick material, fits slim body",
            "Q6" : "Men's lightweight packable down jacket, wind-Resistant, quilted design, snug fit, warm but breathable",
            "Q7" : "Women's high-waisted leggings, tummy control for slimmer look, black color, moisture-wicking fabric, non-see-through",
            "Q8" : "Men's skinny fit stretch jeans, tight on legs but fits good on hips, soft, comfortable and durable",
            "Q9" : "Kid's Rain jacket with hood. Made from waterproof material that keeps dry even in heavy rain. Lightweight but able to keep body warm.",
        }   

In [16]:
for key, query in queries.items():

    # 1) create the embedding for orignal query 
    query_embedding = get_embedding(query)
    print(f"=== {key}: Original Query === {query}")

    # 2) search the FAISS index with orignal query embedding 
    df_result_original, result_idx = search_with_original_query(df_apparel, faiss_index, query_embedding, num_of_records=100)

    # 3) generate opposite query 
    opposite_query = generate_opposite_query(query)
    print(f"=== {key}: Opposite Query === {opposite_query}")

    # 4) get the embedding for the opposite query 
    opposite_query_embedding = get_embedding(opposite_query)

    # 5) search the FAISS index with the opposite query. We need to pass the result_idx from the orignal query in order to limit the seach scope 
    df_result_opposite = search_with_opposite_query(df_apparel, faiss_index, opposite_query_embedding, result_idx, num_of_records=100)

    # 6) generate the reconsile result 
    df_reconcile_result = get_reconcile_result(df_result_original, df_result_opposite)


    # Method 1
    df_result_method1 = get_final_search_result(df_reconcile_result, clipping=0, weight=0, method='average')
    # Method 2
    df_result_method2 = get_final_search_result(df_reconcile_result, clipping=0, weight=0, method='discount_reward')
    # Method 3
    df_result_method3 = get_final_search_result(df_reconcile_result, clipping=0.1, weight=0.5, method='discount_reward')

    # Save results 
    path = '../resources/output/'

    # Uncomment below lines to save the result files. 
    #df_result_method1.to_csv(path + key + "_Method1_Average.tsv", sep='\t', index=False)
    #df_result_method2.to_csv(path + key + "_Method2_Discount_Reward_Only.tsv", sep='\t', index=False)
    #df_result_method3.to_csv(path + key + "_Method3_Discount_Reward_with_Adjustment.tsv", sep='\t', index=False)


=== Q1: Original Query === Long thin cotton socks for men, need to be breathable, even feeling cool for summer time.
=== Q1: Opposite Query ===  Short means having little length. Thick means having a greater than usual measure across. Unbreathable means not allowing air to pass through. Hot means having or giving out a great deal of heat.
=== Q2: Original Query === Wrinkle free chiffon blouse, sleek style, long sleeve, slim fit, with comfortable inside layer
=== Q2: Opposite Query === 
        Wrinkled means having many wrinkles. Clumsy means lacking grace in movement or posture. Short means having little length. Bulky means large and unwieldy. Loose means not fitting closely or tightly.
=== Q3: Original Query === DC or Marvel superhero costumes for adults. Comes with a mask.  Deluxe and high quality. Comfortable to wear. Washable and durable.
=== Q3: Opposite Query ===  Inferior means of lower quality. Basic means of the simplest kind. Uncomfortable means causing discomfort. Unwashabl

In [17]:
df_result_method1.head(3)

,product_id,review_id,star_rating,distance_original,distance_opposite,distance_adjusted,similarity_score,product_similarity_score,product_title,review_body
74,B00M0H4JHQ,R2SU4B0DZSEKYO,4.0,0.104797,0.263660,0.104797,0.211162,0.150999,Women's Casual Waterproof Outdoor Jacket-Diamo...,Good jacket for the price. Not 100% waterproof...
73,B00M0H4JHQ,R2E42WRDY9SSCV,4.0,0.104967,0.267748,0.104967,0.209890,0.150999,Women's Casual Waterproof Outdoor Jacket-Diamo...,My daughter needed a rain resistant jacket for...
77,B00M0H4JHQ,R2HXT2C5BHIP8B,5.0,0.107489,0.247337,0.107489,0.190899,0.150999,Women's Casual Waterproof Outdoor Jacket-Diamo...,Very nice jacket. Wore it to the North Sea in ...


### List of libraries used

In [19]:
session_info.show()